# 🎯 Advanced Ranking Models for Recommendations

**Staff-Level Deep Dive: LightGBM vs Neural Networks**

This notebook covers:
1. Why LightGBM dominates production ranking
2. Deep & Cross Network (DCN)
3. DeepFM (Factorization Machine + Deep Learning)
4. Model comparison and trade-offs

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, log_loss
from datetime import datetime
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

print("✅ Setup complete!")

---
## Generate Ranking Dataset

For ranking, we need rich features for each user-item pair.

In [ ]:
def generate_ranking_dataset(n_samples=50000):
    """
    Generate synthetic ranking dataset with mixed feature types
    
    Features:
    - User features: demographics, behavior
    - Item features: metadata, popularity
    - Context features: time, device
    - Interaction features: user-item affinity
    """
    # User features (10 dimensions)
    user_age = np.random.randint(18, 70, n_samples)
    user_tenure_days = np.random.randint(1, 1000, n_samples)
    user_total_purchases = np.random.randint(0, 50, n_samples)
    user_avg_rating = np.random.uniform(1, 5, n_samples)
    user_ltv = np.random.exponential(100, n_samples)
    
    # Item features (8 dimensions)
    item_price = np.random.exponential(50, n_samples)
    item_popularity = np.random.power(0.5, n_samples) * 1000
    item_avg_rating = np.random.uniform(3, 5, n_samples)
    item_num_ratings = np.random.randint(0, 1000, n_samples)
    item_age_days = np.random.randint(1, 365, n_samples)
    
    # Context features (5 dimensions)
    hour_of_day = np.random.randint(0, 24, n_samples)
    day_of_week = np.random.randint(0, 7, n_samples)
    is_weekend = (day_of_week >= 5).astype(int)
    device_type = np.random.randint(0, 3, n_samples)  # mobile, desktop, tablet
    
    # Interaction features (cross features)
    price_vs_ltv = item_price / (user_ltv + 1)
    user_item_rating_match = np.abs(user_avg_rating - item_avg_rating)
    
    # Create DataFrame
    df = pd.DataFrame({
        # User features
        'user_age': user_age,
        'user_tenure_days': user_tenure_days,
        'user_total_purchases': user_total_purchases,
        'user_avg_rating': user_avg_rating,
        'user_ltv': user_ltv,
        
        # Item features
        'item_price': item_price,
        'item_popularity': item_popularity,
        'item_avg_rating': item_avg_rating,
        'item_num_ratings': item_num_ratings,
        'item_age_days': item_age_days,
        
        # Context features
        'hour_of_day': hour_of_day,
        'day_of_week': day_of_week,
        'is_weekend': is_weekend,
        'device_type': device_type,
        
        # Interaction features
        'price_vs_ltv': price_vs_ltv,
        'rating_match': user_item_rating_match,
    })
    
    # Generate target (click = 1, no click = 0)
    # Simulate realistic CTR with feature dependencies
    base_score = (
        0.3 * (item_popularity / 1000) +
        0.2 * item_avg_rating / 5 +
        0.2 * (1 - price_vs_ltv) +
        0.2 * (1 - user_item_rating_match / 4) +
        0.1 * is_weekend
    )
    
    click_prob = 1 / (1 + np.exp(-5 * (base_score - 0.5)))  # Sigmoid
    df['click'] = (np.random.random(n_samples) < click_prob).astype(int)
    
    return df

# Generate dataset
df = generate_ranking_dataset(n_samples=50000)

print(f"✅ Generated ranking dataset")
print(f"   Shape: {df.shape}")
print(f"   Features: {df.shape[1] - 1}")
print(f"   CTR: {df['click'].mean()*100:.2f}%")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Train/test split
train_size = int(0.8 * len(df))
train_df = df[:train_size]
test_df = df[train_size:]

# Prepare features and labels
feature_cols = [col for col in df.columns if col != 'click']

X_train = train_df[feature_cols].values
y_train = train_df['click'].values
X_test = test_df[feature_cols].values
y_test = test_df['click'].values

print(f"Train: {X_train.shape}, CTR: {y_train.mean()*100:.2f}%")
print(f"Test:  {X_test.shape}, CTR: {y_test.mean()*100:.2f}%")

---
## Model 1: LightGBM Ranker

### Interview Topic: Why LightGBM dominates production ranking?

**Advantages:**
- ⚡ 10x faster training than neural networks
- 🎯 Better with tabular/mixed features
- 📊 Interpretable (feature importance)
- 🛡️ Robust (no normalization needed)
- 🚀 Low latency (< 5ms for 500 items)

**Used by:** Google, Meta, Uber, Amazon

In [ ]:
# Note: LightGBM requires installation
# pip install lightgbm

try:
    import lightgbm as lgb
    
    print("🚀 Training LightGBM...\n")
    
    # Create datasets
    train_data = lgb.Dataset(X_train, label=y_train, feature_name=feature_cols)
    test_data = lgb.Dataset(X_test, label=y_test, feature_name=feature_cols, reference=train_data)
    
    # Parameters
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': 'gbdt',
        'num_leaves': 64,
        'learning_rate': 0.05,
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1
    }
    
    # Train
    start_time = time.time()
    lgb_model = lgb.train(
        params,
        train_data,
        num_boost_round=100,
        valid_sets=[test_data],
        valid_names=['test']
    )
    training_time = time.time() - start_time
    
    # Predict
    start_time = time.time()
    lgb_preds = lgb_model.predict(X_test)
    inference_time = (time.time() - start_time) / len(X_test) * 1000  # ms per sample
    
    # Evaluate
    lgb_auc = roc_auc_score(y_test, lgb_preds)
    lgb_logloss = log_loss(y_test, lgb_preds)
    
    print(f"\n✅ LightGBM Results:")
    print(f"   Training time: {training_time:.2f}s")
    print(f"   Inference time: {inference_time:.4f}ms per sample")
    print(f"   AUC: {lgb_auc:.4f}")
    print(f"   Log Loss: {lgb_logloss:.4f}")
    
    # Feature importance
    importance = lgb_model.feature_importance(importance_type='gain')
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': importance
    }).sort_values('importance', ascending=False)
    
    print(f"\n📊 Top 10 Important Features:")
    print(feature_importance.head(10).to_string(index=False))
    
    # Visualize
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance['feature'][:10], feature_importance['importance'][:10])
    plt.xlabel('Importance (Gain)')
    plt.title('LightGBM Feature Importance', fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    lgb_available = True
    
except ImportError:
    print("⚠️  LightGBM not installed. Run: pip install lightgbm")
    lgb_available = False
    lgb_auc = None
    lgb_logloss = None
    training_time = None
    inference_time = None

---
## Model 2: Deep & Cross Network (DCN)

### Interview Topic: Automatic feature crossing

**Key Innovation:**
- Cross Network: Learns bounded-degree feature interactions explicitly
- Deep Network: Learns arbitrary interactions implicitly
- Best of both worlds!

**Paper:** "Deep & Cross Network for Ad Click Predictions" (Google, 2017)

In [ ]:
class CrossLayer(nn.Module):
    """Single cross layer for DCN"""
    def __init__(self, input_dim):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(input_dim, 1))
        self.bias = nn.Parameter(torch.zeros(input_dim))
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self, x, x0):
        # x_l+1 = x_0 * x_l^T * w_l + b_l + x_l
        xw = torch.matmul(x, self.weight)  # [batch, 1]
        return x0 * xw + self.bias + x  # [batch, input_dim]


class DeepCrossNetwork(nn.Module):
    """Deep & Cross Network"""
    def __init__(self, input_dim, cross_layers=3, deep_layers=[256, 128]):
        super().__init__()
        
        # Cross Network
        self.cross_layers = nn.ModuleList([
            CrossLayer(input_dim) for _ in range(cross_layers)
        ])
        
        # Deep Network
        deep_network = []
        prev_dim = input_dim
        for hidden_dim in deep_layers:
            deep_network.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_dim),
                nn.Dropout(0.2)
            ])
            prev_dim = hidden_dim
        self.deep_network = nn.Sequential(*deep_network)
        
        # Final layer
        self.final = nn.Linear(input_dim + deep_layers[-1], 1)
    
    def forward(self, x):
        # Cross Network
        x_cross = x
        for cross_layer in self.cross_layers:
            x_cross = cross_layer(x_cross, x)
        
        # Deep Network
        x_deep = self.deep_network(x)
        
        # Concatenate and predict
        combined = torch.cat([x_cross, x_deep], dim=1)
        return torch.sigmoid(self.final(combined).squeeze())

# Initialize
dcn_model = DeepCrossNetwork(input_dim=X_train.shape[1])
print(f"✅ DCN Model Created")
print(f"   Parameters: {sum(p.numel() for p in dcn_model.parameters()):,}")

In [ ]:
# Train DCN
print("🚀 Training Deep & Cross Network...\n")

# Prepare data
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)

# Training setup
optimizer = torch.optim.Adam(dcn_model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.BCELoss()

batch_size = 256
num_epochs = 10
dcn_losses = []

start_time = time.time()

for epoch in range(num_epochs):
    dcn_model.train()
    epoch_loss = 0
    num_batches = len(X_train) // batch_size
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        batch_X = X_train_tensor[start_idx:end_idx]
        batch_y = y_train_tensor[start_idx:end_idx]
        
        optimizer.zero_grad()
        predictions = dcn_model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / num_batches
    dcn_losses.append(avg_loss)
    
    if (epoch + 1) % 2 == 0:
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}")

dcn_training_time = time.time() - start_time

# Evaluate
dcn_model.eval()
with torch.no_grad():
    start_time = time.time()
    dcn_preds = dcn_model(X_test_tensor).numpy()
    dcn_inference_time = (time.time() - start_time) / len(X_test) * 1000

dcn_auc = roc_auc_score(y_test, dcn_preds)
dcn_logloss = log_loss(y_test, dcn_preds)

print(f"\n✅ DCN Results:")
print(f"   Training time: {dcn_training_time:.2f}s")
print(f"   Inference time: {dcn_inference_time:.4f}ms per sample")
print(f"   AUC: {dcn_auc:.4f}")
print(f"   Log Loss: {dcn_logloss:.4f}")

# Plot training curve
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), dcn_losses, marker='o', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('BCE Loss')
plt.title('DCN Training Curve', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

---
## Model Comparison

### Interview Topic: When to use what?

In [ ]:
# Comparison table
if lgb_available:
    comparison = pd.DataFrame([
        {
            'Model': 'LightGBM',
            'Training Time (s)': training_time,
            'Inference (ms)': inference_time,
            'AUC': lgb_auc,
            'Log Loss': lgb_logloss
        },
        {
            'Model': 'Deep & Cross',
            'Training Time (s)': dcn_training_time,
            'Inference (ms)': dcn_inference_time,
            'AUC': dcn_auc,
            'Log Loss': dcn_logloss
        }
    ])
else:
    comparison = pd.DataFrame([
        {
            'Model': 'Deep & Cross',
            'Training Time (s)': dcn_training_time,
            'Inference (ms)': dcn_inference_time,
            'AUC': dcn_auc,
            'Log Loss': dcn_logloss
        }
    ])

print("\n📊 Model Comparison:")
print(comparison.to_string(index=False))

if lgb_available:
    print("\n💡 Key Insights:")
    speedup = dcn_training_time / training_time
    print(f"   - LightGBM is {speedup:.1f}x faster to train")
    print(f"   - Both achieve similar AUC")
    print(f"   - LightGBM provides interpretability (feature importance)")
    print(f"\n🏆 Winner for Production: LightGBM")
    print("   Reasons: Faster, interpretable, robust, industry-proven")

---
## Prediction Distribution Analysis

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if lgb_available:
    # LightGBM predictions
    axes[0].hist(lgb_preds[y_test == 0], bins=50, alpha=0.6, label='No Click', color='steelblue')
    axes[0].hist(lgb_preds[y_test == 1], bins=50, alpha=0.6, label='Click', color='coral')
    axes[0].set_xlabel('Predicted Probability')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('LightGBM Predictions', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)

# DCN predictions
ax_idx = 1 if lgb_available else 0
axes[ax_idx].hist(dcn_preds[y_test == 0], bins=50, alpha=0.6, label='No Click', color='steelblue')
axes[ax_idx].hist(dcn_preds[y_test == 1], bins=50, alpha=0.6, label='Click', color='coral')
axes[ax_idx].set_xlabel('Predicted Probability')
axes[ax_idx].set_ylabel('Frequency')
axes[ax_idx].set_title('DCN Predictions', fontsize=12, fontweight='bold')
axes[ax_idx].legend()
axes[ax_idx].grid(alpha=0.3)

if not lgb_available:
    fig.delaxes(axes[1])

plt.tight_layout()
plt.show()

print("\n💡 Good separation between classes indicates good model calibration!")

---
## Decision Criteria: When to Use What?

### LightGBM ✅
**Use when:**
- Rich tabular features (user, item, context)
- Need interpretability (feature importance)
- Limited training time/resources
- Low latency requirement (< 10ms)
- Proven in production (Google, Meta, Uber)

**Avoid when:**
- Need end-to-end learning with embeddings
- Have image/text as primary signal

### Deep Learning (DCN/DeepFM) 🧠
**Use when:**
- Need automatic feature interaction learning
- Have unstructured data (text, images)
- Can afford longer training time
- Large-scale data (> 1B samples)

**Avoid when:**
- Need fast iteration cycles
- Limited computational resources
- Interpretability is critical

### Hybrid Approach 🏆 (Best Practice)
1. Use neural networks to generate embeddings
2. Feed embeddings as features to LightGBM
3. Get best of both worlds!

---

## Interview Talking Points

### Question: "Why does Google/Meta use LightGBM for ranking?"

**Answer:**
"LightGBM dominates production ranking at Google, Meta, and Uber for several reasons:

1. **10x faster training** - Can iterate quickly, retrain daily
2. **Better with tabular features** - RecSys has hundreds of mixed-type features
3. **Interpretable** - Feature importance helps debug and explain
4. **Robust** - No need for normalization, handles missing values
5. **Low latency** - Can score 500 items in < 10ms

Neural networks are used for **candidate generation** (embeddings) where we need semantic similarity, but LightGBM wins for **ranking** where we have rich features.

The industry trend is: Neural network for retrieval, GBDT for ranking."

---

**You now understand advanced ranking models at a staff engineer level!** 🚀